### This computer can only support the inference stage


In [2]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

inference_pipeline = pipeline(
    task=Tasks.auto_speech_recognition,
    model='damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch')

rec_result = inference_pipeline(audio_in='https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/ASR/test_audio/asr_example_zh.wav')
print(rec_result)


2023-09-01 08:10:31,169 - modelscope - INFO - Model revision not specified, use the latest revision: v1.2.1
2023-09-01 08:10:31,465 - modelscope - INFO - initiate model from /home/lihongji/.cache/modelscope/hub/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch
2023-09-01 08:10:31,466 - modelscope - INFO - initiate model from location /home/lihongji/.cache/modelscope/hub/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.
2023-09-01 08:10:31,467 - modelscope - INFO - initialize model from /home/lihongji/.cache/modelscope/hub/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch
2023-09-01 08:10:31,470 - modelscope - WARNING - No preprocessor field found in cfg.
2023-09-01 08:10:31,470 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-09-01 08:10:31,471 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'mod

{'text': '欢迎大家来体验达摩院推出的语音识别模型'}


In [3]:
# online streaming para
import os
import logging
import torch
import soundfile

from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope.utils.logger import get_logger

logger = get_logger(log_level=logging.CRITICAL)
logger.setLevel(logging.CRITICAL)

os.environ["MODELSCOPE_CACHE"] = "./"
inference_pipeline = pipeline(
    task=Tasks.auto_speech_recognition,
    model='damo/speech_paraformer_asr_nat-zh-cn-16k-common-vocab8404-online',
    model_revision='v1.0.6',
    mode="paraformer_streaming"
)

model_dir = os.path.join(os.environ["MODELSCOPE_CACHE"], "damo/speech_paraformer_asr_nat-zh-cn-16k-common-vocab8404-online")
speech, sample_rate = soundfile.read(os.path.join(model_dir, "example/asr_example.wav"))
speech_length = speech.shape[0]

sample_offset = 0
chunk_size = [8, 8, 4] #[5, 10, 5] 600ms, [8, 8, 4] 480ms
stride_size =  chunk_size[1] * 960
param_dict = {"cache": dict(), "is_final": False, "chunk_size": chunk_size}
final_result = ""

for sample_offset in range(0, speech_length, min(stride_size, speech_length - sample_offset)):
    if sample_offset + stride_size >= speech_length - 1:
        stride_size = speech_length - sample_offset
        param_dict["is_final"] = True
    rec_result = inference_pipeline(audio_in=speech[sample_offset: sample_offset + stride_size],
                                    param_dict=param_dict)
    if len(rec_result) != 0:
        final_result += rec_result['text']
        print(rec_result)
print(final_result.strip())

Downloading: 100%|██████████| 10.9k/10.9k [00:00<00:00, 289kB/s]
Downloading: 100%|██████████| 173k/173k [00:00<00:00, 1.23MB/s]
Downloading: 100%|██████████| 55.5k/55.5k [00:00<00:00, 622kB/s]
Downloading: 100%|██████████| 556/556 [00:00<00:00, 345kB/s]
Downloading: 100%|██████████| 91.0/91.0 [00:00<00:00, 68.7kB/s]
Downloading: 100%|██████████| 725/725 [00:00<00:00, 366kB/s]
Downloading: 100%|██████████| 271M/271M [01:55<00:00, 2.45MB/s] 
Downloading: 100%|██████████| 8.28k/8.28k [00:00<00:00, 5.12MB/s]
Downloading: 100%|██████████| 7.90M/7.90M [00:03<00:00, 2.35MB/s]
Downloading: 100%|██████████| 48.7k/48.7k [00:00<00:00, 535kB/s]
Downloading: 100%|██████████| 34.0k/34.0k [00:00<00:00, 775kB/s]


{'text': '欢'}
{'text': '迎大'}
{'text': '家来'}
{'text': '体验'}
{'text': '达摩院'}
{'text': '推出'}
{'text': '的语'}
{'text': '音识'}
{'text': '别模'}
{'text': '型'}
欢迎大家来体验达摩院推出的语音识别模型
